In [ ]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
import root_pandas as rpd
from root_pandas import read_root

#mpl.rcParams['figure.figsize'] = 6.2,3.5
#mpl.rcParams['axes.labelsize'] = 17
#mpl.rcParams['lines.linewidth'] = 3
#mpl.rcParams['lines.markersize'] = 6
#mpl.rcParams['legend.fontsize']= 15



mpl.rcParams.update({'font.size': 18})
mpl.rcParams.update({'legend.fontsize': 20})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    nbeforecut = inputDataframe.shape[0]
    if nbeforecut>0:
        cutDataframe = inputDataframe.query(cut)
        if text:
            print (text, cutDataframe.shape[0], ' (%2.2f '%(100.0*cutDataframe.shape[0]/nbeforecut), '%)')
    return cutDataframe
def applyHadronCuts(fullDataframe,name='default',isMC=False): 
    dataframe = fullDataframe
    print ('Entries before cut ', dataframe.shape[0])
    dataframe.eval('inelasticity = Nu/5.014', inplace=True)
    dataframe.eval('E = z*Nu', inplace=True)
    dataframe.eval('P = sqrt(E*E-0.1396*0.1396)', inplace=True)
    dataframe.eval('T = E-0.1396', inplace=True)
    dataframe.eval('pt2 = pt*pt', inplace=True)

    #dataframe = applyCut(dataframe, 'Q2>1.0 and Q2<1.25', ' 1.0 <Q2<1.25 :')
    #dataframe = applyCut(dataframe, 'Nu>3.5 and Nu<4.0',  ' 3.5 <Nu<4.0')
    dataframe = applyCut(dataframe, 'P <2.5 ', 'P<2.5 ')
    dataframe = applyCut(dataframe, 'P >0.2 ', 'P>0.2 ')

    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    
    
    
    #dataframe = applyCut(dataframe,'(pid==211 & theta_lab_asso>10.0)|(pid_asso==-211 & theta_lab_asso>30 & theta_lab_asso<120)')

    #dataframe = applyCut(dataframe, '(pid==211 & P_asso>0.200) |(pid_asso==-211 & P_asso>0.200) ', 'pi+ P_asso > 200 MeV, pi- P_asso > 300 MeV')
   
    
    
    
    
    return dataframe

def applyElectronCuts(fullDataframe,name='default',isMC=False): 
    dataframe = fullDataframe
    print ('Entries before cut ', dataframe.shape[0])
    dataframe.eval('inelasticity = Nu/5.014', inplace=True)
    #dataframe = applyCut(dataframe, 'Q2>1.0 and Q2<1.25', ' 1.0 <Q2<1.25 :')
    #dataframe = applyCut(dataframe, 'Nu>3.5 and Nu<4.0',  ' 3.5 <Nu<4.0')
    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    return dataframe





In [ ]:
def getRatio(df_A,df_D,df_electron_A,df_electron_D, variable='z',event_cut='Q2>1.0',hadron_cut = ' and P>0',minz=0.05,maxz=1.0,nbins=9):
    
    
    norm_A = df_electron_A.query(event_cut).shape[0]
    norm_D = df_electron_D.query(event_cut).shape[0]
    print ('Number of electron A', norm_A)
    print ('Number of electron D', norm_D)
    
    y_A, x_conditional = np.histogram(df_A.query(event_cut+hadron_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins))
    y_D, x_conditional = np.histogram(df_D.query(event_cut+hadron_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins))
    x_conditional = (x_conditional[1:] + x_conditional[:-1])/2.0
    err_A = np.true_divide(np.sqrt(y_A),y_A)
    err_D = np.true_divide(np.sqrt(y_D),y_D)
    y_A = np.true_divide(y_A,norm_A)
    y_D = np.true_divide(y_D,norm_D)
    ratio_conditional = np.true_divide(y_A,y_D)
    error_conditional = np.multiply(ratio_conditional, np.sqrt(np.power(err_A,2.0) + np.power(err_D,2.0)))
    
    
    return ratio_conditional,error_conditional,x_conditional

## Get data from Ntuples to dataframes

In [ ]:
#this needs to be read in chunks otherwise it uses all memory. (from https://github.com/scikit-hep/root_pandas)
def getdatainChunks(filename,treename):
    dataframe =pd.DataFrame()
    for df in read_root(filename, treename, chunksize=100000) :
        #print df.shape[0]
        dataframe = pd.concat([dataframe,df])
    
    print (dataframe.shape[0])
    return dataframe
    

In [ ]:
df = {}

## GiBUU 

In [ ]:
for target in ['D','C','Fe','Pb']:
    #GiBUU
    print (target)
    print (' hadron')
    df['GiBUU_%s_hadron'%target]= getdatainChunks('GiBUU_SingleHadron_%s.root'%target, '%s_hadron'%target)
    print (' electron')
    df['GiBUU_%s_electron'%target]= getdatainChunks('GiBUU_SingleHadron_%s.root'%target,'%s_electron'%target)
print (df.keys())

In [ ]:
print (df.keys())

In [ ]:
for target in ['D','C','Fe','Pb']:
    print (target)
    df['GiBUU_%s_electron'%target] = applyElectronCuts(df['GiBUU_%s_electron'%target], isMC=True)
    df['GiBUU_%s_hadron'%target] = applyHadronCuts(df['GiBUU_%s_hadron'%target], isMC=True)
    print (' ') 


In [ ]:
df['GiBUU_D_hadron'].hist(figsize=(15, 15),density=True,alpha=0.5,bins=100)
plt.show()

## Apply cuts for trigger dataframes

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target], df['GiBUU_D_hadron'], 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.1,maxz=1.0,nbins=20)
    plt.errorbar(x,r,yerr=err,label='This work,  %s'%target,fmt='o',ms=8)

plt.xlabel('$z_{2}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)


plt.tight_layout()
plt.savefig('Rh.png')
#plt.xlim([0.05,0.5])

#plt.xscale('log')
#plt.savefig('ComparisonWithHermes_Log.png')

plt.show()

## Rh vs z for both pi+ and pi- 

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

ratio = {}

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.05,maxz=1.0,nbins=10)
    
    plt.errorbar(x,r_pip,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.05,maxz=1.0,nbins=10)
    
    plt.errorbar(x,r_pim,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    
    print ('pi+', r_pip)
    print ('pi-', r_pim)
    r = np.true_divide(r_pip,r_pim)
    ratio[target] = r
    
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh.png')

## pi+ data

In [ ]:
x_data = np.array([0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95])
y_data = {}
erry_data  ={}

y_data['pip_C'] = np.array([1.14717 , 1.02146 , 0.920417 , 0.859335 , 0.834309 , 0.791371 , 0.793883 , 0.71396 , 0.670142 , 0.598357])
erry_data['pip_C'] = np.array([0.00334999 , 0.00176576 , 0.00225293 , 0.00270544 , 0.00373737 , 0.00420233 , 0.00581619 , 0.00784954 , 0.0117757 , 0.0197359])
y_data['pip_Fe'] = np.array([1.11582 , 0.892307 , 0.73433 , 0.653604 , 0.605783 , 0.571566 , 0.561484 , 0.491316 , 0.441108 , 0.347698])
erry_data['pip_Fe'] = np.array([0.00305046 , 0.00147941 , 0.00179491 , 0.00207332 , 0.00276241 , 0.00315102 , 0.0046727 , 0.00531373 , 0.00817504 , 0.0112739]);
y_data['pip_Pb'] = np.array([0.939773 , 0.723217 , 0.572263 , 0.49648 , 0.443852 , 0.419778 , 0.412706 , 0.361019 , 0.30504 , 0.221472])
erry_data['pip_Pb'] = np.array([0.00299519 , 0.00143301 , 0.00158777 , 0.00196509 , 0.00227669 , 0.00262181 , 0.00411626 , 0.00458 , 0.00624186 , 0.00797695])


for key in y_data.keys():
    print(len(y_data[key]))
    print(len(x_data))

In [ ]:
x_data_pim = np.array([0.05,0.15,0.25,0.35,0.45,0.55,0.65])#,0.75,0.85])
y_data['pim_C'] = np.array([1.19051 , 1.05789 , 0.994911 , 0.963299 , 0.89357 , 0.877435 , 0.839448 ])#, 0.942727 , 1.1992])
erry_data['pim_C'] = np.array([0.00784357 , 0.00339159 , 0.0045892 , 0.0076076 , 0.0125617 , 0.0212552 , 0.0339285 ])#, 0.0717924 , 0.265256])
y_data['pim_Fe'] = np.array([1.21592 , 0.980147 , 0.871859 , 0.797297 , 0.717917 , 0.678869 , 0.660773 ])#, 0.729836 , 0.608336])
erry_data['pim_Fe'] = np.array([0.00726136 , 0.00284712 , 0.00360744 , 0.00556093 , 0.00889036 , 0.0143047 , 0.0231925])# , 0.045107 , 0.135551])
y_data['pim_Pb'] = np.array([1.13453 , 0.876805 , 0.761002 , 0.666447 , 0.594527 , 0.558697 , 0.587065])# , 0.598514 , 0.682466])
erry_data['pim_Pb'] = np.array([0.00762017 , 0.00291039 , 0.00368793 , 0.00570679 , 0.00937436 , 0.0155114 , 0.0272304 ])#, 0.050011 , 0.181059])

## Comparison to data

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = '#1f77b4'#'red'
colors['Fe'] = '#2ca02c' #'blue'
colors['Pb']= '#d62728'#'black'
#'#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
ratio = {}

for target in ['C','Fe','Pb']:
    #r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
    #                    df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
    #                    minz=0.05,maxz1.0,nbins=20)
    #print(len(x_data))
    #print(len(y_data[target]))
    plt.errorbar(x_data,y_data['pip_%s'%target],yerr=erry_data['pip_%s'%target],color=colors[target],fmt='o',label=target)
    plt.fill_between(x_data, 1.05*y_data['pip_%s'%target],0.95*y_data['pip_%s'%target],color=colors[target],alpha=0.2)
    #plt.errorbar(x,r_pip,yerr=err,label='GiBUU '+'%s'%target,fmt='-',ms=8,color=colors[target],lw=3)
    plt.errorbar(x_data_pim ,y_data['pim_%s'%target],yerr=erry_data['pim_%s'%target],color=colors[target],fmt='o',label=target)
    plt.fill_between(x_data_pim, 1.05*y_data['pim_%s'%target],0.95*y_data['pim_%s'%target],color=colors[target],alpha=0.2)
 
    

minQ2 = 1.0
maxQ2 = 4.0
minNu = 2.2
maxNu = 4.0
plt.title(r'%2.1f$<Q^{2}<$%2.1f GeV$^{2}$, %2.1f$<\nu<$%2.1f GeV, '%(minQ2,maxQ2,minNu,maxNu),fontsize=20)
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False,fontsize=18)
plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh_data_gibuu.png')
plt.savefig('Rh_data_gibuu.pdf')



In [ ]:
fig = plt.figure(figsize=(16,12))


label = {} 
label['C'] = '$^{12}C$'
label['Fe'] = '$^{56}Fe$'
label['Pb'] = '$^{208}Pb$'
fig, axs = plt.subplots(1, 3,sharex=True, sharey=True,figsize=(16,6),gridspec_kw={'hspace': 0, 'wspace':0})

for i,target in enumerate(['C','Fe','Pb']):
    axs[i].errorbar(x_data,y_data['pip_%s'%target],yerr=erry_data['pip_%s'%target],color=colors[target],fmt='o',ms=10,label='$\pi^{+}$')
    axs[i].fill_between(x_data, 1.05*y_data['pip_%s'%target],0.95*y_data['pip_%s'%target],color=colors[target],alpha=0.2)
    #plt.errorbar(x,r_pip,yerr=err,label='GiBUU '+'%s'%target,fmt='-',ms=8,color=colors[target],lw=3)
    axs[i].errorbar(x_data_pim ,y_data['pim_%s'%target],yerr=erry_data['pim_%s'%target],color=colors[target],fmt='s',ms=10,fillstyle='none',label='$\pi^{-}$')
    axs[i].fill_between(x_data_pim, 1.05*y_data['pim_%s'%target],0.95*y_data['pim_%s'%target],color=colors[target],alpha=0.2)
   

    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.05,maxz=1.0,nbins=10)
    
    #axs[i].errorbar(x,r_pip,yerr=err,label='GiBUU $\pi^{+}$',fmt='-',ms=8,color=colors[target])
    axs[i].plot(x,r_pip,'-',color=colors[target],label='GiBUU $\pi^{+}$')

    r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.05,maxz=1.0,nbins=10)
    
    #axs[i].errorbar(x,r_pim,yerr=err, label='GiBUU $\pi^{-}$',fmt='-',ms=8,ls='--',color=colors[target])
 
    axs[i].plot(x,r_pim,'--',color=colors[target],label='GiBUU $\pi^{-}$')

    
    axs[i].xaxis.set_label_text('$z$',fontsize=22)
    axs[i].legend(loc='lower left',ncol=2,frameon=False,fontsize=17)
axs[0].set_title('$^{12}C$',fontsize=19)
axs[1].set_title('$^{56}Fe$',fontsize=19)
axs[2].set_title('$^{208}Pb$',fontsize=19)
axs[0].set_ylim([0.0,1.25])

axs[0].yaxis.set_label_text('$R_{h}$',fontsize=22)

plt.tight_layout()
plt.savefig('Rh_data_gibuu_3panels.png')
plt.savefig('Rh_data_gibuu_3panels.pdf')


In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

ratio = {}

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.05,maxz=1.0,nbins=10)
    
    plt.errorbar(x,r_pip,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    

    print ('pi+', r_pip)
    print ('pi-', r_pim)
    r = np.true_divide(r_pip,r_pim)
    ratio[target] = r
    
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh.png')

In [ ]:
print (ratio.keys())

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    plt.plot(x,ratio[target],color=colors[target],label=target)
    plt.xlabel('$z$',fontsize=22)
    plt.ylabel('Ratio of $\pi^{+}$ to $\pi^{-}$',fontsize=22) 
    plt.legend(loc='best',ncol=3,frameon=False)
    #plt.ylim([0.0,1.3])
    plt.tight_layout()
    plt.savefig('Rh_pimtopip.png')

# momentum

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.2,maxz=2.5,nbins=10,variable='P')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.2,maxz=2.5,nbins=10,variable='P')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    

plt.xlabel('Momentum ',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
#plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh_mmentum.png')

## Transverse momentum

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and z>0.3and z<0.8'), df['GiBUU_D_hadron'].query('pid==211 and z>0.3 and z<0.8'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.01,maxz=1.2,nbins=15,variable='pt2')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211 and z>0.3 and z<0.8'), df['GiBUU_D_hadron'].query('pid==-211 and z>0.3 and z<0.8'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.01,maxz=1.2,nbins=15,variable='pt2')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    

plt.title('0.3<z<0.8',fontsize=22)
plt.xlabel('Transverse momentum squared [GeV$^{2}$]',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
#plt.ylim([0.0,1.3])

#plt.xscale('log')
plt.tight_layout()
plt.savefig('Rh_pt2.png')

## Transverse momentum vs z

In [ ]:
fig, axs = plt.subplots(1, 3,sharex=True, sharey=True,figsize=(16,6),gridspec_kw={'hspace': 0, 'wspace':0})

for i,target in enumerate(['C','Fe','Pb']):
    
    for counter,edges in enumerate([(0.1,0.3),(0.3,0.5), (0.5,0.7),(0.7,.80)]):
        zquery = ' z>%2.2f and z<%2.2f'%(edges[0],edges[1])
        label = '%2.2f<z<%2.2f'%(edges[0],edges[1])
        #zquery = ' z>0.3 and z<0.8'
        r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and' +zquery), df['GiBUU_D_hadron'].query('pid==211 and' + zquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                            minz=0.01,maxz=1.22,nbins=25,variable='pt2')
    
        axs[i].set_title(target)
        axs[i].errorbar(x,r,yerr=err,fmt='-',ms=8,label=label)
        axs[i].legend(loc='best',frameon=False)
        axs[i].xaxis.set_label_text('$p_{T}^{2}$ [GeV$^{2}$]',fontsize=22)
        axs[i].yaxis.set_data_interval(0,1.0)
axs[0].yaxis.set_label_text('$R_{h}$',fontsize=22)
plt.tight_layout()
plt.savefig('Multiplicityratio_pt2_3panels.pdf')

In [ ]:
fig, axs = plt.subplots(1, 3,sharex=True, sharey=True,figsize=(16,6),gridspec_kw={'hspace': 0, 'wspace':0})

for i,target in enumerate(['C','Fe','Pb']):
    
    for counter,edges in enumerate([(2.5,3.0),(3.0,3.5),(3.5,4.0)]):
        nuquery = ' Nu>%2.2f and Nu<%2.2f'%(edges[0],edges[1])
        label = '%2.2f<Nu<%2.2f'%(edges[0],edges[1])
        #zquery = ' z>0.3 and z<0.8'
        r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and' +nuquery), df['GiBUU_D_hadron'].query('pid==211 and' + nuquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                            minz=0.1,maxz=1.0,nbins=15,variable='z')
    
        axs[i].set_title(target)
        axs[i].errorbar(x,r,yerr=err,fmt='-',ms=8,label=label)
        axs[i].legend(loc='best',frameon=False)
        axs[i].xaxis.set_label_text('$p_{T}^{2}$ [GeV$^{2}$]',fontsize=22)
        axs[i].yaxis.set_data_interval(0,1.0)
axs[0].yaxis.set_label_text('$R_{h}$',fontsize=22)
plt.tight_layout()
plt.savefig('Multiplicityratio_pt2_3panels.pdf')

In [ ]:
broadening_pip = []
mass_number = [12,52,208]
mass_number_cubicroot = [2.28, 3.73,5.92]
fig = plt.figure(figsize=(8,6))
broadening_pim= []

selection = 'z>0.5 and z<0.6 and Nu<3 and Nu>2 and Q2<2 and Q2>1'

ref = df['GiBUU_D_hadron'].query('pid==211 and '+selection)['pt2'].mean()
for target in ['C','Fe','Pb']:
    A = df['GiBUU_%s_hadron'%target].query('pid==211 and '+selection)['pt2'].mean()
    print (A-ref)
    #print A
    broadening_pip.append(A-ref)

plt.plot(mass_number_cubicroot,broadening_pip,'o',ms=10,alpha=0.8,label='$\pi^{+}$')


selection = 'z>0.5 and z<0.6 and Nu<4 and Nu>3 and Q2<2 and Q2>1'
broadening_pip= []
ref = df['GiBUU_D_hadron'].query('pid==211 and '+selection)['pt2'].mean()
for target in ['C','Fe','Pb']:
    A = df['GiBUU_%s_hadron'%target].query('pid==211 and '+selection)['pt2'].mean()
    print (A-ref)
    #print A
    broadening_pip.append(A-ref)

plt.plot(mass_number_cubicroot,broadening_pip,'o',ms=10,alpha=0.8,label='3-4')


    
#print broadening
#ref = df['GiBUU_D_hadron'].query('pid==-211 and '+selection)['pt2'].mean()
#for target in ['C','Fe','Pb']:
#    A = df['GiBUU_%s_hadron'%target].query('pid==-211 and '+selection)['pt2'].mean()
#    print A-ref
#    broadening_pim.append(A-ref)
    
#plt.plot(mass_number_cubicroot,broadening_pim,'o',ms=10,alpha=0.8,label='$\pi^{-}$')


plt.title(r'$0.5<z<0.6$ , $2.0<\nu<3.0$',fontsize=22)
plt.ylabel('$p_{T}^{2}$ broadening [GeV$^{2}$]',fontsize=22)
plt.xlabel('Mass number $^{1/3}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.0,0.06])

#plt.xscale('log')
plt.tight_layout()
plt.savefig('Broadening.png')

## Kinetic energy

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211 and P>0.2'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.1,maxz=2.5,nbins=20,variable='T')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    #r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211 and P>0.2'), 
    #                    df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
    #                    minz=0.1,maxz=2.5,nbins=10,variable='T')
    
    #plt.errorbar(x,r,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    

plt.xlabel('Kinetic Energy',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
#plt.ylim([0.0,1.3])

plt.xscale('log')
plt.tight_layout()
plt.savefig('Rh_mmentum.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.85
maxQ2 = 2.40
minNu = 2.2
maxNu = 3.0

event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
print (event_selection)

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.05,maxz=1.0,nbins=10)
    
    plt.errorbar(x,r_pip,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.05,maxz=1.0,nbins=10)
    
    plt.errorbar(x,r_pim,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    
    print ('pi+', r_pip)
    print ('pi-', r_pim)
    print (np.true_divide(r_pip,r_pim))

plt.title(r'%2.2f$<Q^{2}<$%2.2f GeV$^{2}$, %2.2f$<\nu<$%2.2f GeV'%(minQ2,maxQ2,minNu,maxNu),fontsize=22)
    
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.85
maxQ2 = 2.40
minNu = 2.2
maxNu = 3.0

event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
print (event_selection)

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.1,maxz=np.pi,nbins=10,variable='phi_pq')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.1,maxz=np.pi,nbins=10,variable='phi_pq')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    


plt.title(r'%2.2f$<Q^{2}<$%2.2f GeV$^{2}$, %2.2f$<\nu<$%2.2f GeV'%(minQ2,maxQ2,minNu,maxNu))
    
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
#plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.0
maxQ2 = 1.2


#event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
event_selection = 'Q2>%2.2f and Q2<%2.2f'%(minQ2,maxQ2)

print (event_selection)

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.2,maxz=3.0,nbins=20,variable='P')
    
    plt.errorbar(x,r_pip,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    #r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
    #                    df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
    #                    minz=0.2,maxz=3.0,nbins=20,variable='P')
    
    #plt.errorbar(x,r_pim,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    
    ##print 'pi+', r_pip
    ##print 'pi-', r_pim
    #print np.true_divide(r_pip,r_pim)

plt.title(r'%2.2f$<Q^{2}<$%2.2f GeV$^{2}$'%(minQ2,maxQ2))
    
plt.xlabel('$P [GeV]$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.1,2.5])
#plt.xscale('log')
plt.tight_layout()
plt.savefig('Rh.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.0
maxQ2 = 1.2


#event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
event_selection = 'Q2>%2.2f and Q2<%2.2f'%(minQ2,maxQ2)

print (event_selection)

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.2,maxz=3.0,nbins=20,variable='theta_lab')
    
    plt.errorbar(x,r_pip,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    #r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
    #                    df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
    #                    minz=0.2,maxz=3.0,nbins=20,variable='P')
    
    #plt.errorbar(x,r_pim,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    
    #print 'pi+', r_pip
    #print 'pi-', r_pim
   # print np.true_divide(r_pip,r_pim)

plt.title(r'%2.2f$<Q^{2}<$%2.2f GeV$^{2}$'%(minQ2,maxQ2))
    
plt.xlabel('$\theta rad$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.1,2.5])
#plt.xscale('log')
plt.tight_layout()
plt.savefig('Rh.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.85
maxQ2 = 2.40
minNu = 2.2
maxNu = 3.0

#event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
event_selection = 'Q2>%2.2f and Q2<%2.2f'%(minQ2,maxQ2)

print (event_selection)



plt.vlines(0.2, 0,3.0,color='red')

#plt.xscale('log')
plt.hist(df['GiBUU_D_hadron'].query('pid==211 and P>0.2')['T'],bins=50,alpha=0.3,density=2.0)
for target in ['D','C','Fe','Pb']:
    plt.hist(df['GiBUU_%s_hadron'%target].query('pid==211 and P>0.2')['T'],histtype='step',bins=50,alpha=0.8,density=2.0)

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.85
maxQ2 = 2.40
minNu = 2.2
maxNu = 3.0

#event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
event_selection = 'Q2>%2.2f and Q2<%2.2f'%(minQ2,maxQ2)

print (event_selection)

plt.hist(df['GiBUU_D_hadron'].query('pid==211')['theta_lab'],bins=20,alpha=0.3,density=2.0)
for target in ['D','C','Fe','Pb']:
    plt.hist(df['GiBUU_%s_hadron'%target].query('pid==211')['theta_lab'],histtype='step',bins=20,alpha=0.3,density=2.0)